# Правило Ойи и затем классификация/кластеризация c помощью SoftHebb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
data = pd.read_csv("drive/MyDrive/brca.csv")

In [ ]:
print(data.shape)

(569, 32)


In [ ]:
import matplotlib.pyplot as plt
df = data
missing_counts = df.isnull().sum()
missing_counts = missing_counts[missing_counts > 0]  # Оставляем только признаки с пропусками

if missing_counts.empty:
    print("В данных нет пропусков, строить график пропусков не требуется.")
else:
    plt.figure(figsize=(12,6))
    missing_counts.plot(kind='bar', color='skyblue')
    plt.title('Количество пропусков по признакам')
    plt.xlabel('Признаки')
    plt.ylabel('Количество пропусков')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.tight_layout()
    plt.show()


В данных нет пропусков, строить график пропусков не требуется.


In [ ]:
last_col = df.columns[-1]

df[last_col] = df[last_col].map({'M': 1.0, 'B': 0.0})

print(df[last_col].unique())


[0. 1.]


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

class DataLoader:
    def __init__(self, df, target_col, batch_size=32, shuffle=True, device='cpu'):
        """
        :param df: pandas DataFrame с данными
        :param target_col: имя столбца с целевой переменной
        :param batch_size: размер батча
        :param shuffle: перемешивать данные перед каждой эпохой
        :param device: устройство для тензоров ('cpu' или 'cuda')
        """
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.dataset = self.df
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.device = device
        self.num_samples = len(df)
        self.indices = np.arange(self.num_samples)
        self.current_idx = 0
        if self.shuffle:
            np.random.shuffle(self.indices)

    def __iter__(self):
        self.current_idx = 0
        if self.shuffle:
            np.random.shuffle(self.indices)
        return self

    def __next__(self):
        if self.current_idx >= self.num_samples:
            raise StopIteration

        batch_indices = self.indices[self.current_idx:self.current_idx + self.batch_size]
        batch_data = self.df.iloc[batch_indices]

        X_np = batch_data.drop(columns=[self.target_col]).values.astype(np.float32)
        y_np = batch_data[self.target_col].values


        y_np = y_np.astype(np.int64)

        X = torch.from_numpy(X_np).to(self.device)
        y = torch.from_numpy(y_np).to(self.device)

        self.current_idx += self.batch_size
        return X, y

    def __len__(self):
        return (self.num_samples + self.batch_size - 1) // self.batch_size
# --- Пример использования ---

# Предположим, у вас есть DataFrame df и целевой столбец 'target'
target_col = df.columns[-1]  # например, последний столбец

# Разделяем на train и test (например, 80%/20%)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df[target_col])

# Создаём DataLoader для train и test
train_loader = DataLoader(train_df, target_col=target_col, batch_size=1, shuffle=True)
test_loader = DataLoader(test_df, target_col=target_col, batch_size=1, shuffle=False)
print(train_df.shape)
print(test_df.shape)
device = "cuda"
n_components = 10
num_epochs_oj = 30
DATA_SHAPE = 31
eta_oja = 0.01
device = "cuda"
W_oja = 0.01*torch.randn((DATA_SHAPE, n_components), device = device)
# Пример итерации по train_loader
for i in tqdm(range(num_epochs_oj)):
  for i, (img, label) in enumerate(train_loader):
    img = torch.squeeze(img).to(device)  # Перемещаем изображение на устройство
    img = img.reshape(-1,1)  # Преобразуем в одномерный тензор
    x = img/torch.linalg.norm(img)
    y = W_oja.T @ x
    delta_W = eta_oja * (x @ y.T - W_oja @ (y @ y.T))
    W_oja += delta_W

  W_oja = W_oja / torch.linalg.norm(W_oja, axis=0)




(455, 32)
(114, 32)


100%|██████████| 30/30 [00:25<00:00,  1.16it/s]


In [ ]:
print(W_oja.shape)

torch.Size([31, 10])


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import torch.nn.functional as F

import torch
import torch.nn as nn

class SingleLayerNet(nn.Module):
    def __init__(self, num_digits):
        super().__init__()
        self.num_digits = num_digits
        self.fc = nn.Linear(self.num_digits, 1)

    def forward(self, x):
        x = x.view(-1, self.num_digits)
        x = self.fc(x)
        return x


class Layer:
    def __init__(self, num_neuron, num_synapses, eta, base, device, lr_decay="exp", w_ch1=[], w_ch2=[], bias=[]):
        self.num_neuron = num_neuron
        self.num_synapses = num_synapses
        self.neurons = []
        self.eta = eta.reshape(-1,1).to(device)
        self.w_ch1 = w_ch1
        self.w_ch2 = w_ch2
        self.bias = bias
        self.device = device
        self.lr_decay = lr_decay
        self.eta_0 = self.eta
        self.W = (0.1*torch.randn((num_synapses, num_neuron))).to(device)
        self.y_get = []
        self.b_get = []

        self.b = 0.1*torch.zeros((num_neuron,1), device = self.device)
        self.dicts = [{} for i in range(self.num_neuron)]
        self.base = base


    def get_weights(self):
        return self.W

    def get_bias(self):
        return self.b

    def get_vote(self):
        return self.dicts

    def voting_of_neurons(self, x, label):

        u = torch.zeros((self.num_neuron, 1), device=self.device)  # взвешенные суммы нейронов
        y = torch.zeros((self.num_neuron, 1), device=self.device)  # активации нейронов
        x = x / torch.linalg.norm(x)
        x = torch.reshape(x, (-1, 1))
        y = torch.reshape(y, (-1, 1))
        u = torch.reshape(u, (-1, 1))
        u = self.W.T @ x
        y = self.base ** (u + self.b) / torch.sum(self.base ** (u + self.b))
        label = label.item()
        max_activ = torch.argmax(y)  # .to("cpu").numpy() # возвращаем индекс наиболее активного нейрона
        self.dicts[max_activ][label] = self.dicts[max_activ].get(label, 0) + 1

    def get_predict(self, x):

        u = torch.zeros((self.num_neuron, 1), device=self.device)  # взвешенные суммы нейронов
        y = torch.zeros((self.num_neuron, 1), device=self.device)  # активации нейронов
        x = x / torch.linalg.norm(x)
        x = torch.reshape(x, (-1, 1))
        # print(max(self.b) - min(self.b))
        u = self.W.T @ x
        y = self.base ** (u + self.b) / torch.sum(self.base ** (u + self.b))
        a = torch.argmax(y)
        if (self.dicts[a]):
            label = max(self.dicts[a], key=self.dicts[a].get)  # Находим ключ с максимальным значением
        else:
            label = -1
        return label

    def train(self, x):

        u = torch.zeros((self.num_neuron, 1), device=self.device)  # взвешенные суммы нейронов
        y = torch.zeros((self.num_neuron, 1), device=self.device)  # активации нейронов
        x = x / torch.linalg.norm(x)
        x = torch.reshape(x, (-1, 1))
        y = torch.reshape(y, (-1, 1))
        u = torch.reshape(u, (-1, 1))

        u = self.W.T @ x

        # --- старые вычисления
        y = self.base ** (u + self.b) / torch.sum(self.base ** (u + self.b))
        # для графиков ///////////////////////////////////////
        self.y_get.append(y)

        term1 = self.eta.T * (x @ y.T)  # [num_synapses, num_neuron]
        term2 = self.eta.T * self.W * (u.T * y.T)  # [num_synapses, num_neuron]
        delta_W = term1-term2
        self.W = self.W + delta_W

        # self.b = self.b + self.eta * ( y * torch.exp(-self.b)  - 1.0)# - 0.1*self.b) член регуляризации (doesn't work at all)
        # self.b = torch.softmax(self.b, dim = 0)
        # self.b  = torch.clip(self.b, min = -20, max = 20)
        # для графиков ///////////////////////////
        self.b_get.append(self.b)
        return y
    def get_activations(self, x):
        u = torch.zeros((self.num_neuron, 1), device=self.device)  # взвешенные суммы нейронов
        y = torch.zeros((self.num_neuron, 1), device=self.device)  # активации нейронов
        x = x / torch.linalg.norm(x)
        x = torch.reshape(x, (-1, 1))
        y = torch.reshape(y, (-1, 1))
        u = torch.reshape(u, (-1, 1))


        u = self.W.T @ x
        y = self.base ** (u + self.b) / torch.sum(self.base ** (u + self.b))


        return y
    def w_plt1(self):
        return self.w_ch1

    def w_plt2(self):
        return self.w_ch2

    def y_get(self):
        return self.y_get

    def b_get(self):
        return self.b_get

    def w_plt2(self):
        return self.w_ch2

    def lr_update(self):
        norm = torch.linalg.norm(self.W, axis = 0, keepdims = True).reshape(-1,1)
        nc = 1e-10
        self.eta = (self.eta_0 * (pow(torch.abs(norm - torch.ones_like(norm)), 2))+nc).reshape(-1,1)
    def update_lr(self, t):
        if(self.lr_decay == "exp"):
            gamma_exp = 0.4
            self.eta = self.eta_0*np.exp(-gamma_exp*t)
            self.eta = torch.clip(self.eta, a_min = 0.0055, a_max = 1)
        elif(self.lr_decay == "linear"):
            gamma_lin = 0.95
            self.eta = self.eta_0*(gamma_lin)**t
            self.eta = torch.clip(self.eta, a_min = 0.0055, a_max = 1)
        elif(self.lr_decay == "None"):
            self.eta = self.eta


In [ ]:
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

num_neuron = 10
num_synapses = W_oja.shape[1]
eta = 0.15 * torch.ones(num_neuron)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SingleLayerNet(num_digits = num_neuron)
model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
layer = Layer(num_neuron, num_synapses, eta, base = 2.71828, device = device, lr_decay = "None")
norms = []
preds = []
num_epochs = 2
num_epochs_classifier_head = 2
cnt = 0
eps = 0.01
metrics = []
losses = []

for _ in tqdm(range(num_epochs), desc = "Номер эпохи"):
    for i, (img, label) in enumerate(train_loader):
        img = torch.squeeze(img).to(device)  # Перемещаем изображение на устройство
        img = img.reshape(-1)  # Преобразуем в одномерный тензор
        img = W_oja.T @ img
        layer.lr_update()
        y = layer.train(img)
        if(i % 100 == 0):
            norms.append(torch.linalg.norm(layer.get_weights(), axis = 0).mean())
            print(torch.linalg.norm(layer.get_weights(), axis = 0).mean())
print(y)
for img, label in train_loader:
    img = torch.squeeze(img).to(device)
    img = img.reshape(-1)
    img = W_oja.T @ img
    layer.voting_of_neurons(img,label)

list_of_dicts = layer.get_vote() # выводит результаты голосования нейронов
print(list_of_dicts)

model.train()
for _ in tqdm(range(num_epochs_classifier_head), desc = "Номер эпохи"):
    for i, (img, label) in enumerate(train_loader):
        img = torch.squeeze(img).to(device)
        label = label.to(device)
        img = img.reshape(-1)
        img = W_oja.T @ img
        y = layer.get_activations(img)
        y = y/torch.linalg.norm(y)
        optimizer.zero_grad()
        output = model(y)
        loss = criterion(output, label.float().unsqueeze(1)) #label.float()
        loss.backward()
        optimizer.step()

    model.eval()
    test_loss = 0
    correct = 0
    total_samples = 0  # Счетчик всех обработанных примеров

    with torch.no_grad():
        for i, (img, label) in enumerate(test_loader):
            img = torch.squeeze(img).to(device)
            label = label.to(device).float()  # Приводим к float для BCE
            img = img.reshape(-1)
            img = W_oja.T @ img
            y = layer.get_activations(img)
            y = y/torch.linalg.norm(y)

            output = model(y)

            # Рассчитываем loss
            loss = criterion(output, label.float().unsqueeze(1))
            test_loss += loss.item()

            # Рассчитываем предсказания
            preds = torch.sigmoid(output)  # Применяем сигмоиду для получения вероятностей
            preds = (preds > 0.5).long()  # Конвертируем в 0/1
            # print(preds)
            # Обновляем счетчики
            correct += (preds.squeeze() == label.long()).sum().item()
            total_samples += label.size(0)  # Учитываем размер батча

    # 4. Корректный расчет метрик
    test_loss /= len(test_loader)
    accuracy = 100. * correct / total_samples  # Используем total_samples вместо len(test_loader.dataset)
    metrics.append(accuracy)
    losses.append(test_loss)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{total_samples} ({accuracy:.2f}%)\n')




In [ ]:
import numpy as np
import torch
import gc


pred_labels = []
true_labels = []

for img, label in test_loader:
    img = torch.squeeze(img).to(device)
    img = img.reshape(-1)
    img = W_oja.T @ img
    true_labels.append(label)
    pred_labels.append(layer.get_predict(img))



from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


accuracy = accuracy_score(true_labels, pred_labels)

precision = precision_score(true_labels, pred_labels, average='weighted')

recall = recall_score(true_labels, pred_labels, average='weighted')

f1 = f1_score(true_labels, pred_labels, average='weighted')

conf_matrix = confusion_matrix(true_labels, pred_labels)

accuracy = accuracy_score(true_labels, pred_labels)

f1 = f1_score(true_labels, pred_labels, average='weighted')

conf_matrix = confusion_matrix(true_labels, pred_labels)

classification_rep = classification_report(true_labels, pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nConfusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(classification_rep)


In [ ]:
print(list_of_dicts)

[{8: 1, 3: 1}, {1: 127, 2: 12, 3: 2, 8: 3}, {4: 1}, {2: 1, 7: 1}, {8: 6, 7: 5, 9: 2, 2: 1}, {4: 2}, {5: 6, 6: 11, 2: 1, 3: 1, 4: 2}, {4: 16, 9: 41, 8: 7, 7: 1, 2: 5, 5: 1}, {5: 5, 6: 7, 8: 4}, {5: 3, 3: 3, 6: 1}, {4: 2, 8: 2, 6: 7, 5: 1}, {1: 13, 8: 6, 2: 2, 4: 1}, {0: 2}, {5: 9, 8: 28, 7: 6, 2: 12, 3: 5, 6: 1}, {5: 3, 6: 13, 7: 1, 9: 3, 0: 1, 8: 2, 2: 2, 4: 1}, {7: 3, 3: 3, 5: 10}, {3: 26, 7: 21, 8: 4, 9: 8, 4: 4, 5: 3, 6: 1}, {3: 2, 6: 2, 0: 7}, {8: 64, 5: 66, 3: 84, 0: 1, 2: 4}, {5: 4, 2: 4, 1: 1, 3: 2, 9: 1, 7: 1}, {4: 6, 9: 5, 3: 1, 7: 1}, {}, {4: 9, 8: 117, 2: 16, 1: 8, 7: 21, 3: 39, 9: 8, 5: 1}, {0: 27, 9: 33, 8: 14, 7: 83, 4: 1, 3: 3, 5: 2}, {2: 1, 9: 25, 7: 31, 5: 2, 8: 10, 4: 1}, {}, {7: 87, 3: 14, 8: 3, 5: 1, 1: 1, 2: 2}, {4: 11, 8: 2, 3: 8, 5: 2, 7: 2, 6: 1, 1: 1, 9: 1}, {8: 28, 3: 2, 5: 1, 9: 1, 2: 2}, {7: 67, 4: 19, 9: 9, 2: 5, 1: 4}, {6: 71, 4: 13, 9: 5, 8: 1, 2: 2}, {3: 23, 0: 19, 8: 1, 2: 4}, {7: 2, 0: 1, 8: 3, 5: 2, 4: 1}, {9: 3, 3: 12, 8: 4, 5: 3}, {6: 41, 8: 4, 2: 5